In [1]:
from GodotExperimentWrapper import GodotExperimentWrapper
import pandas as pd
import json
import math
import random

def clamp_HdgD(hdg):      
    if hdg > 180:
        return -(360 - hdg)        
    
    if hdg < -180:
        return (360 + hdg )    
    return hdg

def generate_cases(blue_altitude, red_altitude, angle_off, aspect_angle, min_dist=5, max_dist=50, dist_step=0.5):
    
    cases = []
    dist_cases = [x / 10 for x in range(min_dist * 10, max_dist * 10, int(dist_step * 10))]    
       
    for distance in dist_cases:
        
        blue_hdg = clamp_HdgD(aspect_angle)
        red_hdg = clamp_HdgD(blue_hdg + angle_off)                 
        
        case = {
            'agents_config': {
                'blue_agents': {
                    'init_position': {'x': 0.0, 'y': blue_altitude, 'z': 30.0},                    
                    'init_hdg': blue_hdg,                    
                },    
                'red_agents': {
                    'init_position': {'x': 0.0, 'y': red_altitude, 'z': 30.0 - distance},                    
                    'init_hdg': red_hdg,                    
                }                
            }
        }
        cases.append(case)

    return cases


def generate_random_cases(blue_altitude_range, red_altitude_range, angle_off_range, aspect_angle_range):
    blue_alt = random.uniform(blue_altitude_range[0], blue_altitude_range[1])
    red_alt = random.uniform(red_altitude_range[0], red_altitude_range[1])
    angle_off = random.uniform(angle_off_range[0], angle_off_range[1])
    aspect_angle = random.uniform(aspect_angle_range[0], aspect_angle_range[1])

    condition = {
        "blue_alt": blue_alt,
        "red_alt": red_alt,
        "angle_off": angle_off,
        "aspect_angle": aspect_angle
    }

    return condition

config_dict =   {   'EnvConfig' : {
                        'task': 'b_ace_v1',
                        'env_path': 'BVR_AirCombat/bin/B_ACE_v4.exe',
                        'renderize': 1,
                        'experiment_mode'  : 1,
                        'parallel_envs': 1,
                        'max_cycles': 1800,
                        'seed': 1,
                        'port': 12500,
                        'action_repeat': 20,
                        'speed_up': 2000,
                        'action_type': 'Low_Level_Continuous',                        
                        'full_observation': 1,
                        'actions_2d': 0,
                        
                        "RewardsConfig" : 
                        {                                 
                            "mission_factor": 1.0,
                            "missile_fire_factor": -0.1,
                            "missile_no_fire_factor": -0.001,
                            "missile_miss_factor": -0.5,
                            "detect_loss_factor": -0.1,
                            "keep_track_factor": 0.001,
                            "hit_enemy_factor": 3.0,
                            "hit_own_factor": -5.0,
                            "situation_factor": 0.1,
                            "final_team_killed_factor": -5.0,
                            "final_enemy_on_target_factor": -3.0,
                            "final_enemies_killed_factor": 5.0,
                            "final_max_cycles_factor": 3.0
                        }                                
                    },
                    'AgentsConfig' : { 
                        'blue_agents': 
                        {                        
                            'num_agents': 1,
                            'base_behavior': 'wez_eval_target_nez',
                            'init_position': {'x': 0.0, 'y': 25000.0,'z': 30.0},
                            'offset_pos': {	'x': 0.0, 'y': 0.0, 'z': 0.0},
                            'init_hdg': 0.0,                        
                            'target_position': {'x': 0.0,'y': 25000.0,'z': -30.0},                                        
                            'rnd_offset_range':{'x': 0.0,'y': 0.0,'z': 0.0},
                            'rnd_shot_dist_var': 0.0 																						
                        },                            
                        'red_agents':
                        { 
                            'num_agents': 1,
                            'base_behavior' : 'wez_eval_shooter',
                            'init_position': {'x': 0.0,'y': 25000.0,'z': -30.0},
                            'offset_pos': {'x': 0.0,'y': 0.0,'z': 0.0},
                            'init_hdg' : 180.0,                        
                            'target_position': {'x': 0.0,'y': 25000.0,'z': 30.0},                                        
                            'rnd_offset_range':{'x': 0.0,'y': 0.0,'z': 0.0},
                            'rnd_shot_dist_var': 0.0					
                        }                                
                    }                                             
                }                    


In [4]:
env = None
blue_alt_range     = [1000, 50000]
red_alt_range      = [1000, 50000]
angle_off_range    = [-180,  180 ]
aspect_angle_range = [-90, 90]

random.seed(117700)
conditions = []
for i in range(5000):
    conditions.append(generate_random_cases(blue_alt_range,red_alt_range, angle_off_range, aspect_angle_range ))
    
final_results = []

for i,cond in enumerate(conditions): 
    
    cases =  generate_cases(cond['blue_alt'], cond['red_alt'], cond['angle_off'], cond['aspect_angle'])

    experimentConfig = { 'runs_per_case': 1, 'cases' : cases }
    
    # Create the GodotExperimentWrapper
    if env == None:
        config_dict['ExperimentConfig'] = experimentConfig
        env = GodotExperimentWrapper(config_dict)
    else:
        env.send_sim_config(experimentConfig)

    # Run the experiment
    results = env.watch_experiment()

    # Save the experiment results to a file
    #with open('experiment_results.json' , "w") as file:
    #    json.dump(results, file, indent=4)

    # Flatten the list of lists into a single list of dictionaries
    flat_data = [item for sublist in results for item in sublist]
    # Create the DataFrame
    df = pd.DataFrame(flat_data)

    
    item_index = df.loc[df['reds_killed'] == 1].index.max() 
    if not math.isnan(item_index): 
        Rmax = cases[item_index]['agents_config']['red_agents']['init_position']['z'] - 30.0
    else: 
        Rmax = -1
        print("Fail Params", end= "")
    
            
    cond["Rmax"] = Rmax
    final_results.append(cond)
    if (i%10) == 0:
        print(i, end=".")
    else:
        print(".", end="")
    #env.close()
    #env = None

#env.close()
print(final_results)

waiting for remote GODOT connection on port 12811
connection established
Fail Params0..........10..........20..........30..........40..........50..........60..........70.Fail Params.........80..........90..........100..........110..........120........Fail Params..130.Fail Params.........140..........150..........160..........170..........180......Fail Params....190..........200..........210..........220..........230..........240..........250..........260..........270..........280..........290..........300..........310........Fail Params..320..........330..........340..........350..........360..........370..........380..........390..........400..........410..........Fail Params420..........430.........Fail Params.440..........450..........460........Fail Params..470..........480..........490..........500......Fail Params....510..........520..........530..........540..........550..........560..........570..........580........Fail Params..590..........600.........Fail Params.610..........

In [5]:
df =  pd.DataFrame(final_results)
print(df)
df.to_csv('Rnez_5000_v3_2.csv')

          blue_alt       red_alt   angle_off  aspect_angle  Rmax
0      4089.704406  48696.023841 -169.917206    -89.776616  -1.0
1     47508.495520  19810.118244 -158.262383    -89.247290 -16.0
2     30725.297475  22286.515308 -146.946450     71.865646 -18.0
3     21989.466875  24880.669900 -172.641015    -77.774840 -16.0
4      3216.761987  40586.842672   72.019805    -76.525416  -5.0
...            ...           ...         ...           ...   ...
4995  49827.911525  10997.852439  174.725628     23.770543 -30.5
4996  35335.543376  40560.796096   29.025481    -32.878808 -19.0
4997   3641.052422  11259.091414  139.540906     52.151808 -20.5
4998  32425.717921  31381.638307  -37.720543     86.330192 -15.0
4999   8769.262142  12855.243467 -124.683514    -41.081482 -22.5

[5000 rows x 5 columns]


In [6]:
df[df["Rmax"] != -1].describe()


,blue_alt,red_alt,angle_off,aspect_angle,Rmax
count,4917.000000,4917.000000,4917.000000,4917.000000,4917.000000
mean,25797.882226,25160.221029,-0.696381,-0.186163,-19.803844
std,14144.073213,13916.974120,104.365938,52.106706,5.185612
min,1019.731553,1008.124803,-179.835436,-89.994038,-32.000000
25%,13421.906457,12995.131150,-92.614790,-45.778132,-23.500000
50%,25726.899424,25234.371647,-0.888239,-0.458947,-19.500000
75%,38087.430629,37088.190203,88.923558,45.051524,-16.500000
max,49958.870208,49990.782021,179.978968,89.990702,-5.000000
